In [57]:
import torch
torch.cuda.set_device(7)
if torch.cuda.is_available():
    current_gpu = torch.cuda.current_device()
    print(f"Current default GPU index: {current_gpu}")
    print(f"Current default GPU name: {torch.cuda.get_device_name(current_gpu)}")
else:
    print("No GPUs available.")

Current default GPU index: 7
Current default GPU name: NVIDIA A40


# import

In [58]:
# all imports
import torch
# buffer
from collections import defaultdict
import random
from abc import ABC, abstractmethod
from typing import (
    Any,
    Dict,
    Generic,
    Optional,
    List,
    TYPE_CHECKING,
    Set,
    TypeVar,
)

from avalanche.benchmarks.utils import (
    classification_subset,
    AvalancheDataset,
)
from avalanche.models import FeatureExtractorBackbone
# from ..benchmarks.utils.utils import concat_datasets
from avalanche.benchmarks.utils import concat_datasets
from avalanche.training.storage_policy import ReservoirSamplingBuffer, BalancedExemplarsBuffer, ClassBalancedBuffer

from avalanche.training.storage_policy import ParametricBuffer, RandomExemplarsSelectionStrategy, ExemplarsBuffer, ExperienceBalancedBuffer
from avalanche.benchmarks.utils.data_loader import ReplayDataLoader
from avalanche.training.plugins import SupervisedPlugin
from typing import Optional, TYPE_CHECKING

from avalanche.benchmarks.utils import concat_classification_datasets
from avalanche.training.plugins.strategy_plugin import SupervisedPlugin

if TYPE_CHECKING:
    from avalanche.training.templates import SupervisedTemplate, BaseSGDTemplate

# dataset
from avalanche.benchmarks import SplitMNIST, SplitCIFAR100
from avalanche.benchmarks.classic import SplitCIFAR100
from avalanche.benchmarks.classic import SplitCIFAR10
from avalanche.benchmarks.utils.data_loader import GroupBalancedDataLoader, ReplayDataLoader
from avalanche.benchmarks.generators import nc_benchmark, ni_benchmark
from avalanche.benchmarks.generators import filelist_benchmark, dataset_benchmark, \
                                            tensors_benchmark, paths_benchmark

from avalanche.logging import InteractiveLogger, TensorboardLogger, \
    WandBLogger, TextLogger, TensorboardLogger

from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, loss_metrics

from avalanche.training.plugins.checkpoint import CheckpointPlugin, \
    FileSystemCheckpointStorage
from avalanche.training.determinism.rng_manager import RNGManager
from avalanche.training import Naive, CWRStar, Replay, GDumb, \
    Cumulative, LwF, GEM, AGEM, EWC, AR1
from avalanche.models import SimpleMLP
from avalanche.training import Naive, CWRStar, Replay, GDumb, \
    Cumulative, LwF, GEM, AGEM, EWC, AR1
from avalanche.models import SimpleMLP
from avalanche.training.plugins import ReplayPlugin
from types import SimpleNamespace
from avalanche.training.storage_policy import ParametricBuffer, RandomExemplarsSelectionStrategy

# all imports

import torch
import os
from torch import cat, Tensor
from torch.nn import Module
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset, ConcatDataset, TensorDataset
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torchvision import datasets, transforms
import torch.optim.lr_scheduler # ?
from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop, CenterCrop, RandomHorizontalFlip, Resize
from torchvision.transforms.functional import center_crop
from torchvision.models import resnet18, ResNet18_Weights
from torchvision.utils import save_image
from torchvision.transforms.functional import pil_to_tensor


In [59]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import inf


In [60]:
# import torch
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import os

stats = ((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))

transform = transform_train = Compose([
    # Resize(224),
    # Resize(384),
    # RandomHorizontalFlip(),
    ToTensor(),
    # Normalize(*stats,inplace=True)
])

# Load the CIFAR-100 training set
trainset = torchvision.datasets.CIFAR100(root='data', train=True,
                                         download=True, transform=transform)

name_list = trainset.classes

Files already downloaded and verified


# ReplayDataLoader

In [61]:
from avalanche.benchmarks import SplitMNIST
from avalanche.benchmarks.utils.data_loader import GroupBalancedDataLoader, ReplayDataLoader

# Replay Plugin

In [62]:
import random
def sample_Avadataset(dataset, percentage= 0.1):
    """random sample in certain percentage, percentage should be float"""
    data_len = len(dataset)
    indices = list(range(data_len))
    random.shuffle(indices)
    sample_num = int(data_len * percentage)
    sampled_indices = random.sample(indices, sample_num)

    new_buffer_data = dataset.subset(sampled_indices)

    return new_buffer_data




In [63]:
from avalanche.benchmarks.utils.data_loader import ReplayDataLoader
from avalanche.training.plugins import SupervisedPlugin
from typing import Optional, TYPE_CHECKING

from avalanche.benchmarks.utils import concat_classification_datasets
from avalanche.benchmarks.utils.data_loader import ReplayDataLoader
from avalanche.training.plugins.strategy_plugin import SupervisedPlugin
from avalanche.training.storage_policy import (
    ExemplarsBuffer,
    ExperienceBalancedBuffer,
)

if TYPE_CHECKING:
    from avalanche.training.templates import SupervisedTemplate

class CustomReplay(SupervisedPlugin):
    def __init__(
        self,
        mem_size: int = 200,
        batch_size: Optional[int] = None,
        batch_size_mem: Optional[int] = None,
        task_balanced_dataloader: bool = False,
        storage_policy: Optional["ExemplarsBuffer"] = None,
        # The policy that controls how to add new exemplars in memory
                        #
    ):
        super().__init__()
        self.mem_size = mem_size
        self.batch_size = batch_size
        self.batch_size_mem = batch_size_mem
        self.task_balanced_dataloader = task_balanced_dataloader

        self.storage_policy = storage_policy
        assert storage_policy.max_size == self.mem_size



    def before_training_exp(self,
                            strategy: "SupervisedTemplate",
                            num_workers: int = 0,
                            shuffle: bool = True,
                            **kwargs):
        """ Here we set the dataloader. create batch with examples from both
        training data and external memory"""
        if len(self.storage_policy.buffer) == 0:
            # first experience. We don't use the buffer, no need to change
            # the dataloader.
            return

        # replay dataloader samples mini-batches from the memory and current
        # data separately and combines them together.
        print("Override the dataloader.")

        buffer_size = len(self.storage_policy.buffer)
        print("buffer size: " + str(buffer_size))
        num_class = len(self.storage_policy.buffer_datasets)
        print("current class number in replay buffer: " + str(num_class))
        # for item in self.storage_policy.buffer:
        # tempory_buffer = AvalancheDataset([])

        tempory_buffer = []

        for i, dataset in enumerate(self.storage_policy.buffer_datasets):
            sub_data = sample_Avadataset(dataset)
            if i == 0:
                tempory_buffer = sub_data
            else:
                tempory_buffer.concat(sub_data)
                

        buffer = [sample_Avadataset(dataset) for dataset in self.storage_policy.buffer_datasets]
        strategy.dataloader = ReplayDataLoader(
            strategy.adapted_dataset,
            tempory_buffer,
            num_workers=num_workers,
            batch_size=strategy.train_mb_size,
            shuffle=shuffle)

    def after_training_exp(self, strategy: "SupervisedTemplate", **kwargs):
        """ We update the buffer after the experience.
            You can use a different callback to update the buffer in a different place
        """
        print("Buffer update.")
        self.storage_policy.update(strategy, **kwargs)


In [64]:

class ReplayPlugin_us(SupervisedPlugin):
    """
    Experience replay plugin.

    Handles an external memory filled with randomly selected
    patterns and implementing `before_training_exp` and `after_training_exp`
    callbacks.
    The `before_training_exp` callback is implemented in order to use the
    dataloader that creates mini-batches with examples from both training
    data and external memory. The examples in the mini-batch is balanced
    such that there are the same number of examples for each experience.

    The `after_training_exp` callback is implemented in order to add new
    patterns to the external memory.

    The :mem_size: attribute controls the total number of patterns to be stored
    in the external memory.

    :param batch_size: the size of the data batch. If set to `None`, it
        will be set equal to the strategy's batch size.
    :param batch_size_mem: the size of the memory batch. If
        `task_balanced_dataloader` is set to True, it must be greater than or
        equal to the number of tasks. If its value is set to `None`
        (the default value), it will be automatically set equal to the
        data batch size.
    :param task_balanced_dataloader: if True, buffer data loaders will be
            task-balanced, otherwise it will create a single dataloader for the
            buffer samples.
    :param storage_policy: The policy that controls how to add new exemplars
                           in memory
    """

    def __init__(
        self,
        mem_size: int = 200,
        batch_size: Optional[int] = None,
        batch_size_mem: Optional[int] = None,
        task_balanced_dataloader: bool = False,
        storage_policy: Optional["ExemplarsBuffer"] = None,
    ):
        super().__init__()
        self.mem_size = mem_size
        self.batch_size = batch_size
        self.batch_size_mem = batch_size_mem
        self.task_balanced_dataloader = task_balanced_dataloader

        if storage_policy is not None:  # Use other storage policy
            self.storage_policy = storage_policy
            assert storage_policy.max_size == self.mem_size
        else:  # Default
            self.storage_policy = ExperienceBalancedBuffer(
                max_size=self.mem_size, adaptive_size=True
            )

    def before_training_exp(
        self,
        strategy: "SupervisedTemplate",
        num_workers: int = 0,
        shuffle: bool = True,
        drop_last: bool = False,
        **kwargs
    ):
        """
        Dataloader to build batches containing examples from both memories and
        the training dataset
        """
        buffer_size = len(self.storage_policy.buffer)
        print("buffer size: " + str(buffer_size))
        num_class = len(self.storage_policy.buffer_datasets)
        print("current class number in replay buffer: " + str(num_class))
        
        if len(self.storage_policy.buffer) == 0:
            # first experience. We don't use the buffer, no need to change
            # the dataloader.
            return

        batch_size = self.batch_size
        if batch_size is None:
            batch_size = strategy.train_mb_size

        batch_size_mem = self.batch_size_mem
        if batch_size_mem is None:
            batch_size_mem = strategy.train_mb_size

        assert strategy.adapted_dataset is not None

        other_dataloader_args = dict()

        if "ffcv_args" in kwargs:
            other_dataloader_args["ffcv_args"] = kwargs["ffcv_args"]

        if "persistent_workers" in kwargs:
            if parse(torch.__version__) >= parse("1.7.0"):
                other_dataloader_args["persistent_workers"] = kwargs[
                    "persistent_workers"
                ]
        strategy.dataloader = ReplayDataLoader(
                strategy.adapted_dataset,
                self.storage_policy.buffer,
                batch_size=strategy.train_mb_size,
                num_workers=num_workers,
                shuffle=shuffle,
            )

#         strategy.dataloader = ReplayDataLoader(
#             strategy.adapted_dataset,
#             self.storage_policy.buffer,
#             oversample_small_tasks=True,
#             batch_size=batch_size,
#             batch_size_mem=batch_size_mem,
#             task_balanced_dataloader=self.task_balanced_dataloader,
#             num_workers=num_workers,
#             shuffle=shuffle,
#             drop_last=drop_last,
#             **other_dataloader_args

#         )

    def after_training_exp(self, strategy: "SupervisedTemplate", **kwargs):
        self.storage_policy.update(strategy, **kwargs)

In [65]:
# help(ParametricBuffer)

In [66]:
# strategy.dataloader = ReplayDataLoader(
#             strategy.adapted_dataset, #
#             self.storage_policy.buffer,
#             oversample_small_tasks=True,
#             num_workers=num_workers,
#             batch_size=strategy.train_mb_size,
#             shuffle=shuffle)
# self.storage_policy.buffer  determins what is added to memory buffer

In [67]:
# self.storage_policy.update(strategy, **kwargs)-> parametric buffer update

In [68]:
from torch.utils.data.dataset import Subset, ConcatDataset, TensorDataset

class NewParametricBuffer(BalancedExemplarsBuffer):
    """Stores samples for replay using a custom selection strategy and
    grouping."""

    def __init__(
        self,
        max_size: int,
        groupby=None,
        selection_strategy: Optional["ExemplarsSelectionStrategy"] = None,
    ):
        """Init.
        :param max_size: The max capacity of the replay memory.
        :param groupby: Grouping mechanism. One of {None, 'class', 'task',
            'experience'}.
        :param selection_strategy: The strategy used to select exemplars to
            keep in memory when cutting it off.
        """
        super().__init__(max_size)
        assert groupby in {None, "task", "class", "experience"}, (
            "Unknown grouping scheme. Must be one of {None, 'task', "
            "'class', 'experience'}"
        )
        self.groupby = groupby
        ss = selection_strategy or RandomExemplarsSelectionStrategy()
        self.selection_strategy = ss
        self.seen_groups: Set[int] = set()
        self._curr_strategy = None


    def update(self, strategy: "SupervisedTemplate", **kwargs):
        assert strategy.experience is not None
        new_data: AvalancheDataset = strategy.experience.dataset
        new_groups = self._make_groups(strategy, new_data)
        self.seen_groups.update(new_groups.keys())

        # associate lengths to classes
        lens = self.get_group_lengths(len(self.seen_groups))
        group_to_len = {}
        for group_id, ll in zip(self.seen_groups, lens):
            group_to_len[group_id] = ll

        # update buffers with new data
        for group_id, new_data_g in new_groups.items():
            ll = group_to_len[group_id]
            if group_id in self.buffer_groups:
                old_buffer_g = self.buffer_groups[group_id]
                old_buffer_g.update_from_dataset(strategy, new_data_g)
                old_buffer_g.resize(strategy, ll)
            else:
                new_buffer = _ParametricSingleBuffer(ll, self.selection_strategy)
                new_buffer.update_from_dataset(strategy, new_data_g)
                self.buffer_groups[group_id] = new_buffer

        # resize buffers
        for group_id, class_buf in self.buffer_groups.items():
            self.buffer_groups[group_id].resize(strategy, group_to_len[group_id])

    def _make_groups(
        self, strategy, data: AvalancheDataset
    ) -> Dict[int, AvalancheDataset]:
        """Split the data by group according to `self.groupby`."""
        # if self.groupby is None:
        #     return {0: data}
        # elif self.groupby == "task":
        #     return self._split_by_task(data)
        # elif self.groupby == "experience":
        #     return self._split_by_experience(strategy, data)
        # elif self.groupby == "class":
        return self._split_by_class(data)
        # else:
        #     assert False, "Invalid groupby key. Should never get here."

    def _split_by_class(self, data: AvalancheDataset) -> Dict[int, AvalancheDataset]:
        # Get sample idxs per class
        cl_idxs: Dict[int, List[int]] = defaultdict(list)
        targets = getattr(data, "targets")
        for idx, target in enumerate(targets):
            target = int(target)
            cl_idxs[target].append(idx)

        # Make AvalancheSubset per class
        new_groups: Dict[int, AvalancheDataset] = {}
        for c, c_idxs in cl_idxs.items():
            new_groups[c] = classification_subset(data, indices=c_idxs)
        return new_groups

    # def _split_by_experience(
    #     self, strategy, data: AvalancheDataset
    # ) -> Dict[int, AvalancheDataset]:
    #     exp_id = strategy.clock.train_exp_counter + 1
    #     return {exp_id: data}

    # def _split_by_task(self, data: AvalancheDataset) -> Dict[int, AvalancheDataset]:
    #     new_groups = {}
    #     task_set = getattr(data, "task_set")
    #     for task_id in task_set:
    #         new_groups[task_id] = task_set[task_id]
    #     return new_groups



class _ParametricSingleBuffer(ExemplarsBuffer):
    """A buffer that stores samples for replay using a custom selection
    strategy.

    This is a private class. Use `ParametricBalancedBuffer` with
    `groupby=None` to get the same behavior.
    """

    def __init__(
        self,
        max_size: int,
        selection_strategy: Optional["ExemplarsSelectionStrategy"] = None,
    ):
        """
        :param max_size: The max capacity of the replay memory.
        :param selection_strategy: The strategy used to select exemplars to
                                   keep in memory when cutting it off.
        """
        super().__init__(max_size)
        ss = selection_strategy or RandomExemplarsSelectionStrategy()
        self.selection_strategy = ss
        self._curr_strategy = None

    def update(self, strategy: "SupervisedTemplate", **kwargs):
        assert strategy.experience is not None
        new_data = strategy.experience.dataset
        self.update_from_dataset(strategy, new_data)

    def update_from_dataset(self, strategy, new_data):
        # print(new_data[1])
        torch.manual_seed(41)

        data_len = len(new_data)
        indices = torch.randperm(data_len).tolist()
        random.shuffle(indices)
        sample_num = int(data_len * 0.1)
#         sample_num = int(data_len * 0.2)
        sampled_indices = indices[:sample_num]
        # new_buffer_data = new_data[:sample_num]
        # new_buffer_data = Subset(new_data, sampled_indices)
        new_buffer_data = new_data.subset(sampled_indices)

        self.buffer = self.buffer.concat(new_buffer_data)
        self.resize(strategy, self.max_size)

    def resize(self, strategy, new_size: int):
        self.max_size = new_size
        idxs = self.selection_strategy.make_sorted_indices(
            strategy=strategy, data=self.buffer
        )
        self.buffer = self.buffer.subset(idxs[: self.max_size])


In [69]:
l = [1,2,3,4]
random.sample(l, 2)

[3, 1]

# Data transform

In [70]:
# !pip install pytorch_pretrained_vit


import torch
import matplotlib.pyplot as plt
import numpy as np
from torch.nn import CrossEntropyLoss
from torch.optim import SGD

import torch.optim.lr_scheduler # ?

from avalanche.benchmarks.classic import SplitCIFAR100
from avalanche.benchmarks.classic import SplitCIFAR10

from avalanche.benchmarks.generators import nc_benchmark, ni_benchmark

from avalanche.benchmarks.generators import filelist_benchmark, dataset_benchmark, \
                                            tensors_benchmark, paths_benchmark

from torchvision import transforms
from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop, CenterCrop
from torchvision.transforms.functional import center_crop

In [71]:
from torchvision import transforms
from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop, RandomHorizontalFlip, Resize
import os
stats = ((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
transform_train = Compose([
    Resize(224),
    RandomHorizontalFlip(),
    ToTensor(),
    Normalize(*stats,inplace=True)
])

transform_test = Compose([
    Resize(224),
    ToTensor(),
    Normalize(*stats,inplace=True)
])

In [72]:
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.logging import InteractiveLogger, TensorboardLogger, \
    WandBLogger, TextLogger
# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('logs/method1_5epochs_test3.txt', 'w'))

# print to stdout
interactive_logger = InteractiveLogger()

In [73]:
# evaluation
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, loss_metrics

# The evaluation plugin manages the metrics computation.
eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=False, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=False, epoch=True, experience=True, stream=True),
    # forgetting_metrics(experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger])

In [74]:
from avalanche.training.plugins.checkpoint import CheckpointPlugin, \
    FileSystemCheckpointStorage
from avalanche.training.determinism.rng_manager import RNGManager


In [75]:
# RNGManager.set_random_seeds(1234)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

# checkpoint_plugin = CheckpointPlugin(
#     FileSystemCheckpointStorage(
#         directory='./checkpoints/task_cifar',
#     ),
#     map_location=device
# )

# # Load checkpoint (if exists in the given storage)
# # If it does not exist, strategy will be None and initial_exp will be 0
# strategy, initial_exp = checkpoint_plugin.load_checkpoint_if_exists()

cuda


In [76]:
# training
from avalanche.training import Naive, CWRStar, Replay, GDumb, \
    Cumulative, LwF, GEM, AGEM, EWC, AR1

# strategies
from avalanche.models import SimpleMLP
from torch.optim import SGD
from torch.nn import CrossEntropyLoss

# fixed replay 20 tasks-0412

In [77]:
from torchvision.models import resnet18, ResNet18_Weights
weights = ResNet18_Weights.DEFAULT
resnet_model = resnet18(weights=weights)

In [78]:
# training
from avalanche.training import Naive, CWRStar, Replay, GDumb, \
    Cumulative, LwF, GEM, AGEM, EWC, AR1

# strategies
from avalanche.models import SimpleMLP
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.training.plugins import ReplayPlugin

from types import SimpleNamespace

In [79]:
benchmark = SplitCIFAR100(n_experiences=20,
                          train_transform=transform_train,
                          eval_transform = transform_test,
                          seed = 41
                          )

Files already downloaded and verified
Files already downloaded and verified


In [80]:
from avalanche.training.storage_policy import ParametricBuffer, RandomExemplarsSelectionStrategy


In [81]:
storage_p = NewParametricBuffer(
    max_size=60000,
    groupby='class',
    selection_strategy=RandomExemplarsSelectionStrategy()
)


In [82]:
cl_strategy = Naive(
    resnet_model, torch.optim.SGD(resnet_model.parameters(), lr=0.01, momentum = 0.9),
    CrossEntropyLoss(), train_mb_size=32, train_epochs=5, eval_mb_size=16,
    # eval_every=500,
    device=device,
    evaluator=eval_plugin,
    plugins=[ReplayPlugin_us(mem_size=60000, storage_policy = storage_p)]
    )


## 10% replay

In [83]:
# Training
print('Starting experiment...')
results = []
for experience in benchmark.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(benchmark.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [0, 36, 5, 20, 54]
-- >> Start of training phase << --
buffer size: 0
current class number in replay buffer: 0
100%|████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [00:31<00:00,  2.53it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.4205
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.6708
100%|████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [01:39<00:00,  1.27s/it]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.5053
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8488
100%|████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [05:51<00:00,  4.45s/it]
Epoch 2 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.3107
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9136
100%|████████████████████████████████████

	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp017 = 0.0000
-- Starting eval on experience 18 (Task 0) from test stream --
100%|████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [01:10<00:00,  2.19s/it]
> Eval on experience 18 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp018 = 21.0343
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp018 = 0.0000
-- Starting eval on experience 19 (Task 0) from test stream --
100%|████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [01:08<00:00,  2.13s/it]
> Eval on experience 19 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp019 = 17.0698
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp019 = 0.0000
-- >> End of eval phase << --
	Loss_Stream/eval_phase/test_stream/Task000 = 17.4886
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.0467
Start of experience:  1
Current Classes:  [13, 45, 8

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:05<00:00,  5.49it/s]
> Eval on experience 15 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp015 = 20.8520
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp015 = 0.0000
-- Starting eval on experience 16 (Task 0) from test stream --
100%|████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:05<00:00,  6.35it/s]
> Eval on experience 16 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp016 = 23.0809
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp016 = 0.0000
-- Starting eval on experience 17 (Task 0) from test stream --
100%|████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:05<00:00,  6.25it/s]
> Eval on experience 17 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp017 = 31.1156
	Top1

	Loss_Exp/eval_phase/test_stream/Task000/Exp012 = 65.5435
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp012 = 0.0000
-- Starting eval on experience 13 (Task 0) from test stream --
100%|████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:06<00:00,  5.22it/s]
> Eval on experience 13 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp013 = 65.2407
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp013 = 0.0000
-- Starting eval on experience 14 (Task 0) from test stream --
100%|████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:05<00:00,  5.66it/s]
> Eval on experience 14 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp014 = 105.7015
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp014 = 0.0000
-- Starting eval on experience 15 (Task 0) from test stream --
100%|██████████████████████████████████████████████████████████████████

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:05<00:00,  5.74it/s]
> Eval on experience 10 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp010 = 34.5503
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp010 = 0.0000
-- Starting eval on experience 11 (Task 0) from test stream --
100%|████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:03<00:00, 10.01it/s]
> Eval on experience 11 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp011 = 28.9268
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp011 = 0.0000
-- Starting eval on experience 12 (Task 0) from test stream --
100%|████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:05<00:00,  5.37it/s]
> Eval on experience 12 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp012 = 74.5773
	Top1

...

-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:05<00:00,  6.16it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 2.3911
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.5240
-- Starting eval on experience 1 (Task 0) from test stream --
100%|████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:04<00:00,  7.41it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp001 = 4.5138
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.3080
-- Starting eval on experience 2 (Task 0) from test stream --
100%|███████████████████████████████████████████████████████████████████

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 133/133 [00:59<00:00,  2.24it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.3352
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9007
100%|██████████████████████████████████████████████████████████████████████████████████████████████| 133/133 [00:50<00:00,  2.62it/s]
Epoch 2 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.2019
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9440
100%|██████████████████████████████████████████████████████████████████████████████████████████████| 133/133 [00:44<00:00,  3.01it/s]
Epoch 3 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.1215
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9641
100%|██████████████████████████████████████████████████████████████████████████████████████████████| 133/133 [00:49<00:00,  2.70it/s]
Epoch 4 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.0793
	Top1_Acc_Epoc

-- Starting eval on experience 19 (Task 0) from test stream --
100%|████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:03<00:00,  8.12it/s]
> Eval on experience 19 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp019 = 15.4462
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp019 = 0.0000
-- >> End of eval phase << --
	Loss_Stream/eval_phase/test_stream/Task000 = 4.1105
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.4094
Start of experience:  18
Current Classes:  [77, 15, 17, 87, 92]
-- >> Start of training phase << --
buffer size: 4500
current class number in replay buffer: 90
100%|██████████████████████████████████████████████████████████████████████████████████████████████| 141/141 [00:42<00:00,  3.32it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.0190
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8153
100%|█████████████████████████████████████████████████████████████

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:03<00:00, 10.44it/s]
> Eval on experience 16 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp016 = 1.9767
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp016 = 0.5140
-- Starting eval on experience 17 (Task 0) from test stream --
100%|████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:03<00:00,  9.33it/s]
> Eval on experience 17 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp017 = 1.0336
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp017 = 0.7240
-- Starting eval on experience 18 (Task 0) from test stream --
100%|████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:05<00:00,  5.79it/s]
> Eval on experience 18 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp018 = 1.4418
	Top1_Ac

	Loss_Exp/eval_phase/test_stream/Task000/Exp013 = 3.3747
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp013 = 0.3440
-- Starting eval on experience 14 (Task 0) from test stream --
100%|████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:03<00:00,  9.00it/s]
> Eval on experience 14 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp014 = 2.8335
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp014 = 0.4220
-- Starting eval on experience 15 (Task 0) from test stream --
100%|████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:03<00:00,  8.28it/s]
> Eval on experience 15 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp015 = 2.0383
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp015 = 0.5220
-- Starting eval on experience 16 (Task 0) from test stream --
100%|██████████████████████████████████████████████████████████████████████

## 20% replay

In [27]:
# Training
print('Starting experiment...')
results = []
for experience in benchmark.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(benchmark.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [0, 36, 5, 20, 54]
-- >> Start of training phase << --
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [00:12<00:00,  6.23it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.2758
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.7068
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [00:07<00:00, 11.02it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.4317
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8684
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

KeyboardInterrupt: 

## previous

In [ ]:
# Training
print('Starting experiment...')
results = []
for experience in benchmark.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(benchmark.test_stream))

Streaming output truncated to the last 5000 lines.
	Loss_Exp/eval_phase/test_stream/Task000/Exp012 = 20.1502
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp012 = 0.0000
-- Starting eval on experience 13 (Task 0) from test stream --
100%|██████████| 32/32 [00:00<00:00, 39.71it/s]
> Eval on experience 13 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp013 = 20.2334
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp013 = 0.0000
-- Starting eval on experience 14 (Task 0) from test stream --
100%|██████████| 32/32 [00:00<00:00, 37.05it/s]
> Eval on experience 14 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp014 = 19.8389
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp014 = 0.0000
-- Starting eval on experience 15 (Task 0) from test stream --
100%|██████████| 32/32 [00:00<00:00, 37.85it/s]
> Eval on experience 15 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp015 = 20.4529
	Top1_Acc_Exp/eval_phase/test_